In [73]:
%pylab inline
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Populating the interactive namespace from numpy and matplotlib


# AdaGrad

Stochastic Gradient Descent (SGD)는 convex 또는 non-convex function의 optimization에 일반적으로 사용되는 알고리즘입니다.<br>
SGD의 한가지 문제점은 learning rate에 매우 민감합니다. 특히 데이터가 sparse이며 features가 서로다른 frequencies (빈도수)를 갖고 있다면, 단 하나의 learning rate가 모든 weight update에 영향을 주는것은 문제가 될 수 있습니다. 

AgaGrad는 각각의 데이터에 dynamically adapt함으로서 궁극적으로 각각의 feature들마다 서로다른 learning rate로 연산을 합니다.<br>
쉽게 이야기해서, 빈도수가 낮은 데이터에는 높은 learning rate를 적용하고, 빈도수가 높은 데이터에는 낮은 learning rate를 자동으로 적용합니다. <br>
따라서 sparse한 데이터에 적합하며, 대표적인 예가 구글 X lab에서 16000대의 cores를 사용하여 유튜브 동영상안의 고양이를 detection하는 모델에 Adagrad를 사용한것입니다. 

이전의 Momentum update에서는 다음과 같은 공식을 사용했습니다. 

$$ \begin{align}
v &= \gamma v_{t-1} + \eta \nabla_{\theta} J(\theta; x^{(i)},y^{(i)}) \\
\theta &= \theta - v
\end{align} $$

즉 모든 weights $ \theta $에 대해서 동일한 learning rate $ \eta $ 를 사용하여 update를 하였습니다. 

**AdaGrad**에서는 매 다른 step $ t $마다, 각각의 weight $ \theta_i $ 에 대해서, 각각 다른 learning rate를 사용하여 update를 합니다. <br>
먼저 gradient objective function $ g_{t, i} $을 다음과 같이 정의 합니다. 

$$ g_{t, i} = \nabla_\theta J( \theta_i ) $$

각 time step $ t $마다, weights에 대한 업데이트는 다음과 같이 합니다. 

$$ \theta_{t+1, i} = \theta_{t, i} - \eta \cdot g_{t, i} $$ 


### AdaGrad Model

[Deep Learning 책 참고 299page](https://books.google.co.kr/books/about/Deep_Learning.html?id=Np9SDQAAQBAJ&redir_esc=y)

* Global Learning Rate $ \eta $ 가 필요합니다. 
* weights $ \theta $에 대한 초기화가 필요합니다.
* small constant $ \epsilon $ 값을 설정합니다. (numerical stability를 위해서 약 $ 10^{-8} $로 설정, 즉 분모에 들어가게 되는데 0이 안되도록 매우 작은 수를 넣음)
* gradient accumulation variable $ r = 0 $ 초기화합니다.

먼저 gradient를 구한뒤 sqaured gradient (gradient의 제곱)을 r에 accumulation해줍니다.

$$ \begin{align}
g &= \nabla_{\theta} J(\theta; x^{(i)}, y^{(i)}) \\ 
r &= r + g \circledcirc g 
\end{align} $$ 

update를 계산합니다. 

$$ \Delta \theta = - \frac{\eta}{\epsilon + \sqrt{r}} \circledcirc g $$

update를 적용합니다. 

$$ \theta = \theta + \Delta \theta $$

> $ \circledcirc $ 기호는 element wise multiplication 입니다 

### Data

* [Credit Card Fraud Detect - Kaggle](https://www.kaggle.com/dalpozz/creditcardfraud)에서 다운로드 받을수 있습니다. 

In [82]:
data = pd.read_csv('../../data/credit-card-fraud-detection/creditcard.csv')

# Preprocessing Amount
amt_scale = StandardScaler()
data['NormAmount'] =  amt_scale.fit_transform(data['Amount'].values.reshape(-1, 1))

# Split Train and Test Data
X = data.drop(['Time', 'Amount', 'Class'], axis=1).as_matrix()
Y = data['Class'].as_matrix()
# Y = Y.reshape(Y.shape[0], 1)

# Standardization
scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.25, random_state=1)
print('Train X shape:', train_x.shape)
print('Train Y shape:', train_y.shape)
print('Test X shape:', test_x.shape)
print('Test Y shape:', test_y.shape)

Train X shape: (213605, 29)
Train Y shape: (213605,)
Test X shape: (71202, 29)
Test Y shape: (71202,)


### Stochastic Gradient Descent with AdaGrad Optimizer

In [92]:
w = np.random.randn(29 + 1)

def predict(w, x):
    N = len(x)
    yhat = w[1:].dot(x.T) + w[0]
    return yhat

def adagrad_nn(w, X, Y, eta=0.1, ):
    N = len(X)
    e = 1e-8
    r = 0
    for i in range(N):
        x = X[i]
        y = Y[i]
        yhat = predict(w, x)
        gradient = np.sum(-(y - yhat) * x)
        r = r + gradient * gradient
        
        update = -eta/(e + np.sqrt(r)) * gradient
        w = w + update
        
    return w
    

for i in range(1):
    w = adagrad_nn(w, train_x, train_y)
    yhats = predict(w, test_x)
    yhats = np.where(yhats >= 0.5, 1, 0)
    accuracy = accuracy_score(test_y, yhats)
    print(accuracy)

0.45906013876
